In [1]:
import os
import glob
os.chdir('/kaggle/')

In [2]:
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/ 
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

--2022-12-11 19:11:51--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412575 (26M) [application/octet-stream]
Saving to: ‘VnCoreNLP-1.1.1.jar’

VnCoreNLP-1.1.1.jar 100%[===================>]  26.14M   147MB/s    in 0.2s    

2022-12-11 19:11:52 (147 MB/s) - ‘VnCoreNLP-1.1.1.jar’ saved [27412575/27412575]

--2022-12-11 19:11:53--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting respon

In [3]:
!wget https://public.vinai.io/PhoBERT_base_transformers.tar.gz
!tar -xzvf PhoBERT_base_transformers.tar.gz
!wget https://public.vinai.io/PhoBERT_base_fairseq.tar.gz
!tar -xzvf PhoBERT_base_fairseq.tar.gz

--2022-12-11 19:11:59--  https://public.vinai.io/PhoBERT_base_transformers.tar.gz
Resolving public.vinai.io (public.vinai.io)... 54.230.31.74, 54.230.31.76, 54.230.31.104, ...
Connecting to public.vinai.io (public.vinai.io)|54.230.31.74|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322405979 (307M) [application/x-tar]
Saving to: ‘PhoBERT_base_transformers.tar.gz’

PhoBERT_base_transf 100%[===================>] 307.47M  82.5MB/s    in 3.8s    

2022-12-11 19:12:03 (81.1 MB/s) - ‘PhoBERT_base_transformers.tar.gz’ saved [322405979/322405979]

PhoBERT_base_transformers/
PhoBERT_base_transformers/config.json
PhoBERT_base_transformers/bpe.codes
PhoBERT_base_transformers/model.bin
PhoBERT_base_transformers/dict.txt
--2022-12-11 19:12:12--  https://public.vinai.io/PhoBERT_base_fairseq.tar.gz
Resolving public.vinai.io (public.vinai.io)... 54.230.31.104, 54.230.31.76, 54.230.31.108, ...
Connecting to public.vinai.io (public.vinai.io)|54.230.31.104|:443... connected.
H

In [4]:
!pip install transformers
!pip install fastBPE
!pip install fairseq
!pip install vncorenlp

  Preparing metadata (setup.py) ... done
  Created wheel for fastBPE: filename=fastBPE-0.1.0-cp37-cp37m-linux_x86_64.whl size=746401 sha256=b135d4dba60dd6799f3c3254925157981c638165293042f541d2a79c58964823
  Stored in directory: /root/.cache/pip/wheels/bd/d4/0e/0d317a65f77d3f8049fedd8a2ee0519164cf3e6bd77ef886f1
Successfully built fastBPE
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 10.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.4/235.4 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=31a9581bc1af7f410a37bb61a7e90c2439149d9996996c90360f5ac2cca7ef22
  Stored in directory: /

# Dataset

In [5]:
def chunk_text(encoded_text, max_len, max_chunk):
  res = []
  if len(encoded_text) > max_chunk*max_len:
    encoded_text = encoded_text[:max_len*max_chunk] 
  for i in range(0, len(encoded_text), max_len):
    res.append(encoded_text[i:i+max_len])
  k = len(res)
  for i in range(max_chunk-k):
    res.append(torch.zeros(1).long().tolist())
  if len(res) != max_chunk:
    print('value of k:', k)
    print('len(res) in chunk_text:', len(res))
    print('len(encoded_text', len(encoded_text))
  assert len(res) == max_chunk
  return res


In [6]:
from vncorenlp import VnCoreNLP
import torch
from torch.utils.data import Dataset, DataLoader, Subset
from tensorflow.keras.preprocessing.sequence import pad_sequences
from fairseq.data.encoders.fastbpe import fastBPE
from fairseq.data import Dictionary
from torchvision import  transforms
import json
import random
import argparse
from PIL import Image
import pandas as pd
import numpy as np
import cv2

MAX_LEN = 256
MAX_CHUNKS = 8

class FoodyDataset(Dataset):
  def __init__(self, data_dir, path, test=False):
    self.data_dir = data_dir
    self.path = path
    self.test = test
    if not test:
      self.data_list = self._load_json()
    else:
      self.data_list = self._load_csv()
    self.rdrsegmenter = VnCoreNLP("./vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 
    parser = argparse.ArgumentParser()
    parser.add_argument('--bpe-codes', 
                        default="./PhoBERT_base_transformers/bpe.codes",
                        required=False,
                        type=str,
                        help='path to fastBPE BPE')
    args, unknown = parser.parse_known_args()
    self.bpe = fastBPE(args)
    self.vocab = Dictionary()
    self.vocab.add_from_file("./PhoBERT_base_transformers/dict.txt")

    self.transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
                                                                                                                                                                
  def __len__(self):
    return len(self.data_list)

  def __getitem__(self, idx):
    s = self.data_list[idx]
    
    revid = s['RevId']
    text = str(s['Comment'])
    text = self.rdrsegmenter.tokenize(text)
    text = ' '.join([' '.join(x) for x in text])
    subwords = '<s> ' + self.bpe.encode(text) + ' </s>'
    encoded_text = self.vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long().tolist()
    encoded_text = chunk_text(encoded_text, MAX_LEN, MAX_CHUNKS)
    chunked_tokens = pad_sequences(encoded_text, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
    masks = [[int(token_id > 0) for token_id in sentence] for sentence in chunked_tokens]
    if not self.test:
      img_path = os.path.join(self.data_dir, 'image', str(revid))
    else:
      img_path = os.path.join(self.data_dir, 'image_test', str(revid))

    img_path = os.path.join(img_path, '0.jpg')
    img = self._load_image(img_path)
    if not self.test:
      label = int(s['Rating'] >= 6.0)
      return torch.tensor(chunked_tokens).long(), torch.tensor(masks).long(), img, torch.tensor(label).long()
    return revid, torch.tensor(chunked_tokens).long(), torch.tensor(masks).long(), img

  def _load_json(self):
    with open(self.path, 'r') as f:
      dataset = json.load(f)
    return dataset

  def _load_csv(self):
    df = pd.read_csv(self.path)
    dataset = []
    for _, row in df[['RevId', 'Comment']].iterrows():
      dataset.append({'RevId': row['RevId'], 
                      'Comment': row['Comment']})
    return dataset
  
  def _load_image(self, path):
    img = cv2.imread(path)
    if img is None:
      print(path)
      img = np.zeros((1000,1000,3))
    img = Image.fromarray(img.astype('uint8'), 'RGB')

    img_tensor = self.transform(img)
    return img_tensor

def build_dataloader(dataset,
                      train_ids=None,
                      val_ids=None,
                      batch_size=4,
                      num_workers=1,
                      device='cuda',
                      test=False,
                    ):
  if not test:
    train_set = Subset(dataset, list(train_ids))
    val_set = Subset(dataset, list(val_ids))

    train_loader = DataLoader(train_set, 
                              batch_size=batch_size,
                              num_workers=num_workers,
                              shuffle=True)
    val_loader = DataLoader(val_set, 
                            batch_size=batch_size,
                            num_workers=num_workers,
                            shuffle=False)
    dataloader = (train_loader, val_loader)
  else:
    dataloader = DataLoader(dataset, 
                            batch_size=batch_size,
                            num_workers=num_workers,
                            shuffle=False)
  return dataloader


# Model

In [7]:
import torch
import torch.nn as nn
from torchvision import models
from transformers import RobertaForSequenceClassification, RobertaConfig, AdamW

class FoodyPhoBERTEncoder(nn.Module):
  def __init__(self, out_features=1):
    super(FoodyPhoBERTEncoder, self).__init__()
    self.out_features = out_features
    self.bert_config = RobertaConfig.from_pretrained("./PhoBERT_base_transformers/config.json", 
                                                      from_tf=False, 
                                                      num_labels = out_features, 
                                                     output_hidden_states=False)
    self.bert = RobertaForSequenceClassification.from_pretrained("./PhoBERT_base_transformers/model.bin", 
                                                                 config=self.bert_config)
    self.lstm = nn.LSTM(MAX_CHUNKS*out_features, out_features, batch_first=True)
  def forward(self, tokens, mask, labels, train=True):
    x = 0
    for i in range(MAX_CHUNKS):
      out = self.bert(
          tokens[:,i,:].squeeze().long(),
          token_type_ids=None,
          attention_mask=mask[:,i,:].squeeze().long()
      )
      if i==0:
        x = out[0]
      else:
        x = torch.cat((x, out[0]), dim=-1)
    x = x.unsqueeze(1)
    x, _ = self.lstm(x)
    x = x.squeeze()
    return x

class FoodyConvEncoder(nn.Module):
  def __init__(self, out_features=1, img_model='resnet', dropout=0.1):
    super(FoodyConvEncoder, self).__init__()
    self.out_features = out_features
    self.img_model = img_model
    self.dropout = dropout
    if img_model == 'resnet':
      self.convnet = models.resnet18(pretrained=True)
      n_fts = self.convnet.fc.in_features
      self.convnet.fc = nn.Linear(n_fts, out_features)
    elif img_model == 'vgg':
      self.convnet = models.vgg11(pretrained=True)
      self.convnet.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, out_features),
        )
  def forward(self, x):
    x = self.convnet(x)
    return x

class FoodyJointModel(nn.Module):
  def __init__(self, out_features=1, img_model='vgg', dropout=0.1, coef=0.2):
    super(FoodyJointModel, self).__init__()
    self.out_features = out_features
    self.coef = coef
    self.img_model = img_model
    self.dropout = dropout
    
    self.bert_encode = FoodyPhoBERTEncoder(out_features)
    self.conv_encode = FoodyConvEncoder(out_features=out_features, img_model=img_model, dropout=0.1)
    self.linear = nn.Linear(out_features, 1)
    

  def forward(self, x_img, x_tokens, mask, labels=None, train=True):
    out_bert = self.bert_encode(x_tokens, mask, labels, train)
    out_conv = self.conv_encode(x_img)
    out = self.coef * out_conv + (1 - self.coef) * out_bert
    out = self.linear(out)
    return out

# Loss Function

In [8]:
import torch.nn.functional as F

def sigmoid_focal_loss(
    inputs: torch.Tensor,
    targets: torch.Tensor,
    alpha: float = 0.25,
    gamma: float = 2,
    reduction: str = "mean",
):
    p = torch.sigmoid(inputs)
    ce_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction="none")
    p_t = p * targets + (1 - p) * (1 - targets)
    loss = ce_loss * ((1 - p_t) ** gamma)

    if alpha >= 0:
        alpha_t = alpha * targets + (1 - alpha) * (1 - targets)
        loss = alpha_t * loss

    if reduction == "mean":
        loss = loss.mean()
    elif reduction == "sum":
        loss = loss.sum()

    return loss

def np_sigmoid(x):
  return 1/(1 + np.exp(-x))

# Train and Test

In [9]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from tqdm import tqdm, tqdm_notebook
import pandas as pd 
import numpy as np


# config

device = 'cuda'
batch_size = 4
out_features = 8
img_model='vgg'
dropout=0.1
coef=0.2
epochs = 10
state_dict_max = 0

model = FoodyJointModel(out_features, 
                        img_model, 
                        dropout,
                        coef)
model.to(device)

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5, correct_bias=False)

kf = KFold(n_splits=10, shuffle=True)
full_dataset = FoodyDataset('./input/foodyvnu/Data','./input/foodyvnu/Data/train.json')
print(len(full_dataset))
test_dataset = FoodyDataset('./input/foodyvnu/Data', './input/foodyvnu/Data/test.csv', test=True)
test_dataloader = build_dataloader(test_dataset,
                                    batch_size=batch_size,
                                    num_workers=1,
                                    device=device,
                                    test=True)

You are using a model of type bert to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at ./PhoBERT_base_transformers/model.bin were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenc

  0%|          | 0.00/507M [00:00<?, ?B/s]

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
Loading codes from ./PhoBERT_base_transformers/bpe.codes ...
Read 64000 codes from the codes file.


9071


Loading codes from ./PhoBERT_base_transformers/bpe.codes ...
Read 64000 codes from the codes file.


In [10]:
def save_model(model, optimizer, path):
    state_dict = {
        "optimizer": optimizer.state_dict(),
        "model": model.state_dict()
    }
    torch.save(state_dict, path)

In [11]:
for epoch_i, (tid, vid) in enumerate(kf.split(full_dataset)):
  print('======== Partition {:} / {:} ========'.format(epoch_i + 1, epochs))
  train_dataloader, val_dataloader = build_dataloader(full_dataset,
                                                      train_ids=tid,
                                                      val_ids=vid,
                                                      batch_size=batch_size,
                                                      num_workers=1,
                                                      device=device,
                                                      test=False)
  print(len(train_dataloader))
  print(len(val_dataloader))
  print('Training...')
  
  total_loss = 0
  train_logits, train_labels = 0, 0
  print(model.img_model)
  model.train()

  for step, batch in tqdm_notebook(enumerate(train_dataloader), total=len(train_dataloader), desc='Training'):
    batch = tuple(b.to(device).float() for b in batch)
    tokens, masks, img, labels = batch
    
    model.zero_grad()
    outputs = model(
        img,
        tokens,
        masks,
        labels,
        train=True
    )    

    loss = sigmoid_focal_loss(outputs.squeeze(), labels)
    total_loss += loss.item()

    logits = outputs.detach().cpu().numpy()
    labels = labels.to('cpu').numpy()
    if step == 0:
      train_logits = logits.flatten()
      train_labels = labels.flatten()
    else:
      train_logits = np.concatenate([train_logits, logits.flatten()])
      train_labels = np.concatenate([train_labels, labels.flatten()])
    
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()

  avg_train_loss = total_loss / len(train_dataloader)
  print(" ROC-AUC Socre: {0:.4f}".format(roc_auc_score(train_labels, train_labels, average='macro')))
  print(" Average training loss: {0:.4f}".format(avg_train_loss))

  print('Running validation ...')
  model.eval()
  eval_logits, eval_labels = 0, 0

  for step, batch in tqdm_notebook(enumerate(val_dataloader), total=len(val_dataloader), desc='Validation'):

      batch = tuple(t.to(device) for t in batch)

      tokens, masks, img, labels = batch

      with torch.no_grad():
          outputs = model(
              img,
              tokens,
              masks,
              labels,
              train=False
          )
          logits = outputs.detach().cpu().numpy()
          labels = labels.to('cpu').numpy()

          if step == 0:
            eval_logits = logits.flatten()
            eval_labels = labels.flatten()
          else:
            eval_logits = np.concatenate([eval_logits, logits.flatten()])
            eval_labels = np.concatenate([eval_labels, labels.flatten()])
  print([round(float(rate)*10, 1) for rate in np_sigmoid(eval_logits)])
  print(" ROC-AUC score: {0:.4f}".format(roc_auc_score(eval_labels, np_sigmoid(eval_logits), average='macro')))

  print("Running test....")
  model.eval()
  test_rid, test_logits = 0, 0
  for step, batch in tqdm_notebook(enumerate(test_dataloader), total=len(test_dataloader), desc="Testing: "):
      batch = tuple(t.to(device) for t in batch)

      revids, tokens, masks, img = batch

      with torch.no_grad():
          outputs = model(
              img,
              tokens,
              masks,
              train=False
          )    
          logits = outputs.detach().cpu().numpy()
          revids = revids.detach().cpu().numpy()
          if step == 0:
            test_logits = logits.flatten()
            test_rid = revids.flatten()
          else:
            test_logits = np.concatenate([test_logits, logits.flatten()])
            test_rid = np.concatenate([test_rid, revids.flatten()])
  results = {
      'RevId': [int(id) for id in test_rid],
      'Rating': [round(float(rate)*10, 1) for rate in np_sigmoid(test_logits)]
  }
  df = pd.DataFrame(results)
  save_model(model, optimizer, f"./working/state_dict_{epoch_i}.pth")
  df.to_csv(f'./working/submit_joint_{epoch_i}.csv')



======== Partition 1 / 10 ========
2041
227
Training...
vgg


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


Training:   0%|          | 0/2041 [00:00<?, ?it/s]

 ROC-AUC Socre: 1.0000
 Average training loss: 0.0422
Running validation ...


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:56: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

[9.2, 9.4, 10.0, 10.0, 10.0, 9.9, 9.4, 9.5, 7.2, 9.3, 10.0, 9.9, 10.0, 9.3, 0.4, 10.0, 8.4, 9.6, 0.1, 9.8, 9.2, 9.8, 9.7, 9.9, 9.5, 10.0, 10.0, 10.0, 0.0, 10.0, 9.9, 7.6, 10.0, 9.9, 10.0, 9.9, 0.2, 10.0, 9.8, 10.0, 9.3, 10.0, 9.8, 10.0, 9.9, 10.0, 9.8, 8.8, 0.2, 10.0, 10.0, 9.5, 9.9, 8.8, 0.1, 10.0, 0.0, 0.2, 10.0, 10.0, 9.7, 0.2, 9.8, 0.0, 0.0, 10.0, 8.4, 9.3, 9.9, 7.9, 9.9, 10.0, 10.0, 9.5, 8.9, 9.3, 10.0, 9.7, 0.0, 9.9, 0.2, 10.0, 10.0, 10.0, 8.7, 9.8, 10.0, 10.0, 0.0, 10.0, 10.0, 9.7, 10.0, 8.4, 9.8, 10.0, 8.4, 9.7, 9.9, 10.0, 10.0, 9.6, 10.0, 9.8, 0.1, 10.0, 0.0, 0.0, 10.0, 9.8, 10.0, 10.0, 9.7, 9.9, 10.0, 10.0, 10.0, 9.2, 10.0, 10.0, 0.3, 10.0, 9.8, 9.7, 9.8, 10.0, 0.0, 9.7, 10.0, 9.9, 8.0, 9.6, 0.1, 0.2, 0.0, 10.0, 10.0, 10.0, 0.2, 0.0, 10.0, 10.0, 10.0, 10.0, 9.9, 1.5, 9.7, 10.0, 9.7, 0.2, 9.6, 9.9, 10.0, 9.8, 10.0, 10.0, 10.0, 10.0, 10.0, 9.9, 9.8, 10.0, 9.6, 9.9, 10.0, 9.6, 10.0, 9.7, 10.0, 10.0, 10.0, 9.9, 10.0, 10.0, 9.6, 0.1, 6.6, 10.0, 9.9, 10.0, 0.0, 0.2, 9.9, 10.0, 9.9,

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:85: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


Testing:   0%|          | 0/1276 [00:00<?, ?it/s]

./input/foodyvnu/Data/image_test/1672433/0.jpg
./input/foodyvnu/Data/image_test/6279731/0.jpg


RuntimeError: [enforce fail at inline_container.cc:300] . unexpected pos 1846360256 vs 1846360160